In [ ]:
import pyarrow
import pyarrow.parquet as pq
from pathlib import Path
import pandas as pd

In [ ]:
data_path = Path("./files")
matched_result_file = Path("matched_result_manual_revised.final.v2.csv")
matched_result = pd.read_csv(matched_result_file)
matched_result = matched_result[matched_result["is_matched"] == "y"]
matched_result = matched_result[["IBES_id", "final_parent_factset_name", "final_parent_factset_id", "ticker", "cusip", "sic"]]
parquet_files = list(data_path.glob("0*.parquet"))
result_dir = Path("./position_data")

In [ ]:
# show how many row groups are in the parquet file
for parquet_file in parquet_files:
    output_file = result_dir / parquet_file.name.replace(".parquet", ".csv")
    if output_file.exists():
        continue
    print("processing ", parquet_file)
    data = pq.ParquetFile(parquet_file)
    n_row_groups = data.num_row_groups
    all_columns = data.schema.names
    # exclude `companyurl`
    if "companyurl" in all_columns:
        all_columns.remove("companyurl")
    all_merged_data = []
    for ri in range(n_row_groups):
        print("processing row group ", ri, " out of ", n_row_groups)
        table = data.read_row_group(ri, columns=all_columns).to_pandas()
        new_marged_table = pd.merge(table, matched_result, on="final_parent_factset_id", how="inner")
        all_merged_data.append(new_marged_table)
    all_merged_data = pd.concat(all_merged_data)
    all_merged_data.to_csv(result_dir / parquet_file.name.replace(".parquet", ".csv"), index=False)
        
        

In [ ]:
all_position_data = list(result_dir.glob("*.csv"))
# find all positions at NY
all_data = []
for position in all_position_data:
    df = pd.read_csv(position)
    df = df[df["state"] == "NY"]
    all_data.append(df)
    print(position, len(df))
all_data = pd.concat(all_data)

In [ ]:
all_data.to_csv("ny_positions.csv", index=False)

# Summarize the dataset

* Brokers in Positions: How many brokers (based on IBES_id) are found in the positions dataset?
* Position Count per Broker: For each broker, how many positions are recorded?
* Unique Users per Broker: How many unique users (user_id) are there for each broker?
* New York Users per Broker: Specifically for each broker, how many unique users are based in New York? Additionally, what percentage does this represent relative to all users and to all US users?

In [ ]:
def summarize_data(data):
    gdata = data[["IBES_id", "position_id", "user_id"]]
    # group by IBES_id
    gdata = gdata.groupby("IBES_id").agg({"position_id": "count", "user_id": "nunique"}).reset_index()
    # rename columns
    gdata = gdata.rename(columns={"position_id": "number_of_positions", "user_id": "number_of_users"})
    # find US users and NY users
    data_us_users = data[data["country"] == "United States"]
    data_ny_users = data_us_users[data_us_users["state"] == "NY"]
    # get number of users grouped by IBES_id
    number_of_us_users = data_us_users.groupby("IBES_id").agg({"user_id": "nunique"}).reset_index()
    # rename column to avoid confusion
    number_of_us_users = number_of_us_users.rename(columns={"user_id": "number_of_users_US"})
    number_of_ny_users = data_ny_users.groupby("IBES_id").agg({"user_id": "nunique"}).reset_index()
    number_of_ny_users = number_of_ny_users.rename(columns={"user_id": "number_of_users_NY"})
    # merge number of users
    gdata = gdata.merge(number_of_us_users, on="IBES_id", how="left")
    gdata = gdata.merge(number_of_ny_users, on="IBES_id", how="left")
    gdata = gdata.fillna(0)
    # set datatype to int
    gdata["number_of_users_US"] = gdata["number_of_users_US"].astype(int)
    gdata["number_of_users_NY"] = gdata["number_of_users_NY"].astype(int)
    return gdata

In [ ]:
all_result_files = list(result_dir.glob("*.csv"))
all_summary = []
for result_file in all_result_files:
    print("processing ", result_file)
    df = pd.read_csv(result_file)
    df_summary = summarize_data(df)
    df_summary["file_name"] = result_file.name
    all_summary.append(df_summary)
all_summary = pd.concat(all_summary)
# group by IBES_id
all_summary = all_summary.groupby("IBES_id").agg({"number_of_positions": "sum", "number_of_users": "sum", "number_of_users_US": "sum", "number_of_users_NY": "sum"}).reset_index()
# remove columns named "file_name"

In [17]:
all_summary["percentage_of_NY_users_in_US"] = all_summary["number_of_users_NY"] / all_summary["number_of_users_US"]

In [19]:
all_summary.to_csv("summary_broker_position_data.csv", index=False)